In [ ]:
import smote_variants as sv
import os
import pandas as pd
import numpy as np
from sklearn.tree import DecisionTreeClassifier
import statistics
from sklearn.metrics import roc_auc_score
from sklearn import preprocessing
from sklearn.preprocessing import LabelEncoder
from imblearn.over_sampling import SMOTE
from collections import Counter
import math
from sklearn.cluster import KMeans  
from sklearn import svm

In [ ]:
# 

path = "/Users/emily/Desktop/Research/oversampling_python/data/"
folderName = 'yeast1-5-fold'#'abalone19-5-fold' # yeast6-5-fold'#'haberman-5-fold' #'abalone19-5-fold' # pima-5-fold yeast-2_vs_8-5-fold glass5-5-fold segment0-5-fold yeast-5-fold yeast1-5-fold


os.chdir(path+ folderName)
dirs = os.listdir(path+ folderName)
train = []
test = []

for i in dirs:
    #print(i.split("-")[-1])
    if("xlsx" in i):
        if("tra" in i):
            train.append(i)

        elif("tst" in i):
            test.append(i)
train = sorted(train)
test = sorted(test)

In [ ]:
fil = pd.read_excel(test[0])
dd = pd.DataFrame(fil)
s = dd.columns[-1]
dd[s]

In [ ]:
def find_maj(sample_class):
    counter = Counter(sample_class);
    maj = list(dict(counter.most_common(1)).keys())
    maj = "".join(maj)
    print(maj)
    return  maj


def classprocess(output):
    c = Counter(output)
    datagap = []
    maj = find_maj(output)
    maj_num = dict(c)[find_maj(output)]
    for className, number in c.items(): 
        #print(className," ",number)
        print(number)
        temp = np.array([className,(maj_num - number)])
        datagap.append(temp)
    return datagap

In [ ]:
# 純 polynom_fit_SMOTE

alloverSMOTE = []
overSMOTE = []
accuracies=[]
#print(os.getcwd())
for ii,i in enumerate(train):
    randomIndex = []
    data = pd.read_excel(i,index_col=0)
    lastColumn = data.columns[-1]

    data[lastColumn]= data[lastColumn].str.replace("\n", "").str.strip()
    l = data.shape[1] -1
    output = data.iloc[:,l];
    
    finaldata = data.iloc[:,:l]
    le = preprocessing.LabelEncoder()
    finaldata.iloc[:,0] = le.fit_transform(finaldata.iloc[:,0])
    output = le.fit_transform(output)
    
    tempover = []
    
    #over = SMOTE()  # SMOTE
    #over = sv.polynom_fit_SMOTE() #  polynom_fit_SMOTE
    #over = sv.SMOTE_IPF() # SMOTE_IPF()
    over = sv.ProWSyn() # ProWSyn()
    finaldata = np.array(finaldata)
    output = np.array(output)
    #X_polynom,y_polynom = over.fit_resample(finaldata,output)
    X_polynom,y_polynom = over.sample(finaldata,output)
  
    clf=svm.SVC(kernel='rbf',C=1,gamma='auto')
    clf = clf.fit(X_polynom,y_polynom)
    
    #clf = clf.fit(finaldata,output)
    #newDataCount = len(X_smote) - len(data)  # 新生成的 data 數量
    
   
    test_file = pd.read_excel(test[ii],index_col=0) #不然會有多出來的 unnamed column
    test_data = pd.DataFrame(test_file);
    test_data[lastColumn]= test_data[lastColumn].str.replace("\n", "").str.strip()   

    """
    # le = preprocessing.LabelEncoder()
    # for i in range(test_data.shape[1]):
    #     test_data.iloc[:,i] = le.fit_transform(test_data.iloc[:,i]) 
    """
    test_X = test_data.iloc[:,:(test_data.shape[1])-1] # 劃分
    test_X.iloc[:,0] = le.fit_transform(test_X.iloc[:,0])
    test_y_predicted = clf.predict(test_X)
    test_y = test_data.iloc[:,test_data.shape[1]-1] 
   
    test_y = le.fit_transform(test_y)
    test_y_predicted = le.fit_transform(test_y_predicted)
    
    accuracy = roc_auc_score(test_y, test_y_predicted)
    accuracies.append(accuracy)
    

mean = statistics.mean(accuracies)

print(mean)




Cluster 

In [1095]:
# Cluster
# polynom_fit_SMOTE
alloverpolynom = []
overpolynom = []

centerpolynom = []
countfor = 0;
for ii,i in enumerate(train):
    randomIndex = []
    le = preprocessing.LabelEncoder()
    data = pd.read_excel(i,index_col=0)
    lastColumn = data.columns[-1]

    data[lastColumn]= data[lastColumn].str.replace("\n", "").str.strip()
    originlen = data.shape[0]
    output = data.iloc[:,data.shape[1]-1];
    classCount = classprocess(output)
    finaldata = data.iloc[:,:data.shape[1]-1]

    output = le.fit_transform(output)
    finaldata.iloc[:,0] = le.fit_transform(finaldata.iloc[:,0])
    
    #output.iloc[:] = le.fit_transform(output.iloc[:])

    tempover = []
   
    finaldata = np.array(finaldata)
    output = np.array(output)
    over = sv.polynom_fit_SMOTE()
    
    X_polynom,y_polynom = over.sample(finaldata,output)
    newDataCount = len(X_polynom) - len(data)  # 新生成的 data 數量
    # 把 X_polynom 跟 y_polynom 和在一起
    X_polynom = pd.DataFrame(X_polynom)
    y_polynom = pd.DataFrame(y_polynom)
    alloverpolynom = pd.concat([X_polynom,y_polynom],axis=1) # SMOTE 完後的數據
    
    overpolynom.append(alloverpolynom)

    for i in range(len(classCount)):
        countfor = math.floor(int(classCount[i][1])*0.4); # 要產生多少數據  無條件捨去
        #randomIndex.extend([random.randint(len(data),len(X_smote)-1) for _ in range(count)]) 
        
        if(countfor>0):
            kmeans = KMeans(n_clusters=1)
            dtemp = pd.DataFrame(overpolynom[ii])
            X = dtemp.iloc[originlen:,:dtemp.shape[1]-1] # 後來生成的
            
            kmeans.fit(X)
            y_kmeans = kmeans.predict(X)
            centers = kmeans.cluster_centers_
            
            distance = []
            X = X.astype('float64')
            centers = centers.astype('float64')
            tempindata = {}
            distancesortemp = []
            for i in range(X.shape[0]-1): # 列
                
                distance = []
                temp = 0;
                for j in range(X.shape[1]-1):#9 行
                    temp = pow((centers[0][j]-X.iloc[i][j]),2)  
                    tempindata[i] = temp
            
            distancesortemp = sorted(tempindata.items(), key=lambda item:item[1])
    
     
            centerpolynom.append(distancesortemp[:countfor])

    

print(len(centerpolynom[0])) # 第一份資料中的群中心數量

2021-05-31 10:21:57,588:INFO:polynom_fit_SMOTE: Running sampling via ('polynom_fit_SMOTE', "{'proportion': 1.0, 'topology': 'star', 'random_state': None}")
negative
negative
844
343
2021-05-31 10:21:58,149:INFO:polynom_fit_SMOTE: Running sampling via ('polynom_fit_SMOTE', "{'proportion': 1.0, 'topology': 'star', 'random_state': None}")
negative
negative
844
343
2021-05-31 10:21:58,717:INFO:polynom_fit_SMOTE: Running sampling via ('polynom_fit_SMOTE', "{'proportion': 1.0, 'topology': 'star', 'random_state': None}")
negative
negative
844
343
2021-05-31 10:21:59,325:INFO:polynom_fit_SMOTE: Running sampling via ('polynom_fit_SMOTE', "{'proportion': 1.0, 'topology': 'star', 'random_state': None}")
negative
negative
844
343
2021-05-31 10:21:59,906:INFO:polynom_fit_SMOTE: Running sampling via ('polynom_fit_SMOTE', "{'proportion': 1.0, 'topology': 'star', 'random_state': None}")
negative
negative
844
344
200


In [1096]:
# centerpolynom 只是 index ，value 是取出值
centerpolynomvalue =[]
for tr in train:
    data = pd.read_excel(tr,index_col=0)
    originlen = len(data)
    for i in range(len(centerpolynom)):
        alltemp = []
        for j in range(len(centerpolynom[i])):
            indexpolynom = centerpolynom[i][j][0] + originlen
            #tempSMOTE = list(overSMOTE[i][indexSMOTE])
            alltemp.append(list(overpolynom[i].iloc[indexpolynom]))
        centerpolynomvalue.append(alltemp)

In [1097]:
# Cluster
# ProWSyn
alloverProWSyn = []
overProWSyn = []

centerProWSyn = []
countfor = 0;
for ii,i in enumerate(train):
    randomIndex = []
    le = preprocessing.LabelEncoder()
    data = pd.read_excel(i,index_col=0)
    lastColumn = data.columns[-1]

    data[lastColumn]= data[lastColumn].str.replace("\n", "").str.strip()
    originlen = data.shape[0]
    output = data.iloc[:,data.shape[1]-1];
    classCount = classprocess(output)
    finaldata = data.iloc[:,:data.shape[1]-1]

    output = le.fit_transform(output)
    finaldata.iloc[:,0] = le.fit_transform(finaldata.iloc[:,0])
    
    #output.iloc[:] = le.fit_transform(output.iloc[:])

    tempover = []
    print("origin",Counter(output))
    finaldata = np.array(finaldata)
    output = np.array(output)
    over = sv.ProWSyn()
    
    X_ProWSyn,y_ProWSyn = over.sample(finaldata,output)
    print(Counter(y_ProWSyn))
    newDataCount = len(X_polynom) - len(data)  # 新生成的 data 數量
    # 把 X_polynom 跟 y_polynom 和在一起
    X_ProWSyn = pd.DataFrame(X_ProWSyn)
    y_ProWSyn = pd.DataFrame(y_ProWSyn)
    alloverProWSyn = pd.concat([X_ProWSyn,y_ProWSyn],axis=1) # SMOTE 完後的數據
    
    overProWSyn.append(alloverProWSyn)

    for i in range(len(classCount)):
        countfor = math.floor(int(classCount[i][1])*0.3); # 要產生多少數據  無條件捨去
        #randomIndex.extend([random.randint(len(data),len(X_smote)-1) for _ in range(count)]) 
        
        if(countfor>0):
            kmeans = KMeans(n_clusters=1)
            dtemp = pd.DataFrame(overProWSyn[ii])
            X = dtemp.iloc[originlen:,:dtemp.shape[1]-1] # 後來生成的
            
            kmeans.fit(X)
            y_kmeans = kmeans.predict(X)
            centers = kmeans.cluster_centers_
            
            distance = []
            X = X.astype('float64')
            centers = centers.astype('float64')
            tempindata = {}
            distancesortemp = []
            for i in range(X.shape[0]-1): # 列
                
                distance = []
                temp = 0;
                for j in range(X.shape[1]-1):#9 行
                    temp = pow((centers[0][j]-X.iloc[i][j]),2)  
                    tempindata[i] = temp
            
            distancesortemp = sorted(tempindata.items(), key=lambda item:item[1])
    
     
            centerProWSyn.append(distancesortemp[:countfor])

    

print(len(centerProWSyn[0])) # 第一份資料中的群中心數量

2021-05-31 10:22:02,901:INFO:ProWSyn: Running sampling via ('ProWSyn', "{'proportion': 1.0, 'n_neighbors': 5, 'L': 5, 'theta': 1.0, 'n_jobs': 1, 'random_state': None}")
negative
negative
844
343
origin Counter({0: 844, 1: 343})
Counter({0: 844, 1: 844})
2021-05-31 10:22:03,685:INFO:ProWSyn: Running sampling via ('ProWSyn', "{'proportion': 1.0, 'n_neighbors': 5, 'L': 5, 'theta': 1.0, 'n_jobs': 1, 'random_state': None}")
negative
negative
844
343
origin Counter({0: 844, 1: 343})
Counter({0: 844, 1: 844})
2021-05-31 10:22:04,409:INFO:ProWSyn: Running sampling via ('ProWSyn', "{'proportion': 1.0, 'n_neighbors': 5, 'L': 5, 'theta': 1.0, 'n_jobs': 1, 'random_state': None}")
negative
negative
844
343
origin Counter({0: 844, 1: 343})
Counter({0: 844, 1: 844})
2021-05-31 10:22:05,213:INFO:ProWSyn: Running sampling via ('ProWSyn', "{'proportion': 1.0, 'n_neighbors': 5, 'L': 5, 'theta': 1.0, 'n_jobs': 1, 'random_state': None}")
negative
negative
844
343
origin Counter({0: 844, 1: 343})
Counter({0

In [1098]:
# centerProWSyn
centerProWSynvalue =[]
for tr in train:
    data = pd.read_excel(tr,index_col=0)
    originlen = len(data)
    for i in range(len(centerProWSyn)):
        alltemp = []
        for j in range(len(centerProWSyn[i])):
            indexProWSyn = centerProWSyn[i][j][0] + originlen
            #tempSMOTE = list(overSMOTE[i][indexSMOTE])
            alltemp.append(list(overProWSyn[i].iloc[indexProWSyn]))
        centerProWSynvalue.append(alltemp)

In [ ]:
# 合併 小類 center polynom + ProWSyn 
allCenterHalf = []
temp = []
for i in range(len(centerpolynom)):
    temp = centerpolynomvalue[i] + centerProWSynvalue[i]
    temp = pd.DataFrame(temp,columns=data.columns)
    allCenterHalf.append(temp)

len(allCenterHalf[1])

In [1099]:
# Cluster
# SMOTE-IPF
alloverSMOTEIPF = []
overSMOTEIPF = []

centerSMOTEIPF = []
countfor = 0;
for ii,i in enumerate(train):
    randomIndex = []
    le = preprocessing.LabelEncoder()
    data = pd.read_excel(i,index_col=0)
    lastColumn = data.columns[-1]

    data[lastColumn]= data[lastColumn].str.replace("\n", "").str.strip()
    originlen = data.shape[0]
    output = data.iloc[:,data.shape[1]-1];
    classCount = classprocess(output)
    finaldata = data.iloc[:,:data.shape[1]-1]

    output = le.fit_transform(output)
    finaldata.iloc[:,0] = le.fit_transform(finaldata.iloc[:,0])
    
    #output.iloc[:] = le.fit_transform(output.iloc[:])

    tempover = []
   
    finaldata = np.array(finaldata)
    output = np.array(output)
  #output.iloc[:] = le.fit_transform(output.iloc[:])

    tempover = []
   
    finaldata = np.array(finaldata)
    output = np.array(output)
    over = sv.SMOTE_IPF()
    
    X_SMOTEIPF,y_SMOTEIPF = over.sample(finaldata,output)
    newDataCount = len(X_polynom) - len(data)  # 新生成的 data 數量
    # 把 X_polynom 跟 y_polynom 和在一起
    X_SMOTEIPF = pd.DataFrame(X_SMOTEIPF)
    y_SMOTEIPF = pd.DataFrame(y_SMOTEIPF)
    alloverSMOTEIPF = pd.concat([X_SMOTEIPF,y_SMOTEIPF],axis=1) # SMOTE 完後的數據
    
    overSMOTEIPF.append(alloverSMOTEIPF)

    for i in range(len(classCount)):
        countfor = math.floor(int(classCount[i][1])*0.3); # 要產生多少數據  無條件捨去
        #randomIndex.extend([random.randint(len(data),len(X_smote)-1) for _ in range(count)]) 
        
        if(countfor>0):
            kmeans = KMeans(n_clusters=1)
            dtemp = pd.DataFrame(overSMOTEIPF[ii])
            X = dtemp.iloc[originlen:,:dtemp.shape[1]-1] # 後來生成的
            
            kmeans.fit(X)
            y_kmeans = kmeans.predict(X)
            centers = kmeans.cluster_centers_
            
            distance = []
            X = X.astype('float64')
            centers = centers.astype('float64')
            tempindata = {}
            distancesortemp = []
            for i in range(X.shape[0]-1): # 列
                
                distance = []
                temp = 0;
                for j in range(X.shape[1]-1):#9 行
                    temp = pow((centers[0][j]-X.iloc[i][j]),2)  
                    tempindata[i] = temp
            
            distancesortemp = sorted(tempindata.items(), key=lambda item:item[1])
    
     
            centerSMOTEIPF.append(distancesortemp[:countfor])

    

print(len(centerSMOTEIPF[0])) # 第一份資料中的群中心數量 

2021-05-31 10:22:08,653:INFO:SMOTE_IPF: Running sampling via ('SMOTE_IPF', "{'proportion': 1.0, 'n_neighbors': 5, 'n_folds': 9, 'k': 3, 'p': 0.01, 'voting': 'majority', 'n_jobs': 1, 'classifier': DecisionTreeClassifier(random_state=2), 'random_state': None}")
2021-05-31 10:22:08,654:INFO:SMOTE: Running sampling via ('SMOTE', "{'proportion': 1.0, 'n_neighbors': 5, 'n_jobs': 1, 'random_state': <module 'numpy.random' from '/Users/emily/Desktop/Research/oversampling_python/venv/lib/python3.7/site-packages/numpy/random/__init__.py'>}")
2021-05-31 10:22:08,726:INFO:SMOTE_IPF: Removing 0 elements
2021-05-31 10:22:08,785:INFO:SMOTE_IPF: Removing 0 elements
2021-05-31 10:22:08,842:INFO:SMOTE_IPF: Removing 0 elements
negative
negative
844
343
2021-05-31 10:22:09,580:INFO:SMOTE_IPF: Running sampling via ('SMOTE_IPF', "{'proportion': 1.0, 'n_neighbors': 5, 'n_folds': 9, 'k': 3, 'p': 0.01, 'voting': 'majority', 'n_jobs': 1, 'classifier': DecisionTreeClassifier(random_state=2), 'random_state': None}

In [1100]:
# centerSMOTEIPF 只是 index ，value 是取出值
centerSMOTEIPFvalue =[]
for tr in train:
    data = pd.read_excel(tr,index_col=0)
    originlen = len(data)
    for i in range(len(centerSMOTEIPF)):
        alltemp = []
        for j in range(len(centerSMOTEIPF[i])):
            indexSMOTEIPF = centerSMOTEIPF[i][j][0] + originlen
            #tempSMOTE = list(overSMOTE[i][indexSMOTE])
            alltemp.append(list(overSMOTEIPF[i].iloc[indexSMOTEIPF]))
        centerSMOTEIPFvalue.append(alltemp)

In [ ]:
# 合併 小類 center polynom-fit-SMOTE + SMOTE-IPF

allCenterHalf = []
temp = []
for i in range(len(centerpolynom)):
    temp = centerpolynomvalue[i] + centerSMOTEIPFvalue[i]
    temp = pd.DataFrame(temp,columns=data.columns)
    allCenterHalf.append(temp)

len(allCenterHalf[1])

In [ ]:

# 合併 小類 center ProWSyn  + SMOTE-IPF

allCenterHalf = []
temp = []
for i in range(len(centerProWSyn )):
    temp = centerProWSynvalue[i] + centerSMOTEIPFvalue[i]
    temp = pd.DataFrame(temp,columns=data.columns)
    allCenterHalf.append(temp)

len(allCenterHalf[1])

In [1101]:
# 合併 polynom ProWSyn SMOTE_IPF 三個
allCenterHalf = []
temp = []
for i in range(len(centerProWSyn )):
    temp = centerProWSynvalue[i] + centerSMOTEIPFvalue[i]
    temp = temp + centerpolynomvalue[i]
    temp = pd.DataFrame(temp,columns=data.columns)
    allCenterHalf.append(temp)

len(allCenterHalf[1])

500

In [ ]:
allCenterHalf[0]

In [1102]:
# # 跟原始資料合併
from sklearn import preprocessing
from sklearn.preprocessing import LabelEncoder
mergeRandom = []
accuracies = []
for index,element in enumerate(train):
    data = pd.read_excel(element,index_col =0);
    lastColumn = data.columns[-1]

    data[lastColumn]= data[lastColumn].str.replace("\n", "").str.strip()
    l = data.shape[1]-1
    le = preprocessing.LabelEncoder()
    data.iloc[:,l] = le.fit_transform(data.iloc[:,l])
    data.iloc[:,0] = le.fit_transform(data.iloc[:,0])
    
    """
    output = data.iloc[:,l];
    finaldata = data.iloc[:,:l]
    finaldata.iloc[:,0] = le.fit_transform(finaldata.iloc[:,0])
    """
    #data.iloc[:,0] = le.fit_transform(data.iloc[:,0])
    #classCount = classprocess(output)
    #data = data.T

    #allCenterminHalf[index] = pd.DataFrame(allCenterminHalf[index],columns=data.columns)
    mergeRandom = pd.concat([data,allCenterHalf[index]],axis=0)
    
    finaldata = mergeRandom.iloc[:,:l]
    output = mergeRandom.iloc[:,l]
    print(Counter(output))
    clf=svm.SVC(kernel='rbf',C=1,gamma='auto')

    clf = clf.fit(finaldata,output)


    test_file = pd.read_excel(test[index],index_col=0) #不然會有多出來的 unnamed column
    test_data = pd.DataFrame(test_file);
    #test_data.Class= test_data.Class.str.replace("\n", "").str.strip()   

    test_X = test_data.iloc[:,:(test_data.shape[1])-1] 
   
    test_X.iloc[:,0] = le.fit_transform(test_X.iloc[:,0])
    
    
    #output.iloc[:] = le.fit_transform(output.iloc[:])

    test_y_predicted = clf.predict(test_X)

    test_y = test_data.iloc[:,test_data.shape[1]-1] 

    test_y = le.fit_transform(test_y)
    test_y_predicted = le.fit_transform(test_y_predicted)

    accuracy = roc_auc_score(test_y, test_y_predicted)
    accuracies.append(accuracy)
    

mean = statistics.mean(accuracies)
print(mean)

#len(mergeRandom[0][0])


Counter({0.0: 844, 1.0: 843})
Counter({0.0: 844, 1.0: 843})
Counter({0.0: 844, 1.0: 843})
Counter({0.0: 844, 1.0: 843})
Counter({0.0: 844, 1.0: 844})
0.5606873010418759
